In [1]:
import os
import sys
sys.path.append('..')

from utils.dataiter import Dataset
from utils.preprocessing import *
from models.baseline.random_model import random_prediction_model
from models.model.FFNN_ALL import FFNN_ALL
from core.config import raw_features
from utils.evaluate import calculate_ctr, compute_rce, average_precision_score

import core.config as conf
from tqdm import tqdm

## predict

In [5]:
all_features_to_idx = dict(zip(raw_features, range(len(raw_features))))
float_formatter = "{:.15f}".format

def parse_input_line(line):
    features = line.split("\x01")
    tweet_id = features[all_features_to_idx['tweet_id']]
    user_id = features[all_features_to_idx['engager_id']]
    input_feats = features[all_features_to_idx['text_tokens']]
    return tweet_id, user_id, input_feats



In [7]:
def evaluate_test_set():
    path = '/hdd/twitter/raw_lzo/' # ./test
    part_files = sorted([os.path.join(path, f) for f in os.listdir(path) if 'part' in f])[:1]
    model_path = '/hdd/models/ffnn_pkl/'
    ds = Dataset()
    with open('../submission/results.csv', 'w') as output:
        for file in tqdm(part_files):
            df = read_data(file)
            df = ds.pickle_matching(ds.preprocess(df, TARGET_id=conf.REPLY))

            pred_reply = FFNN_ALL(df, conf.REPLY).predict(model_path) 
            pred_retweet = FFNN_ALL(df, conf.RETWEET).predict(model_path) 
            pred_comment = FFNN_ALL(df, conf.COMMNET).predict(model_path) 
            pred_like = FFNN_ALL(df, conf.LIKE).predict(model_path) 

            with open(file, 'r') as f:
                for i, line in enumerate(f.readlines()):
                    tweet_id, user_id, features = parse_input_line(line)
                    reply_pred = pred_reply[i][0]
                    retweet_pred = pred_retweet[i][0]
                    quote_pred = pred_comment[i][0]
                    fav_pred = pred_like[i][0]
                    output.write(f'{tweet_id},{user_id},{reply_pred},{retweet_pred},{quote_pred},{fav_pred}\n')

            del df

In [8]:
evaluate_test_set()

  0%|          | 0/1 [00:00<?, ?it/s]['reply', 'retweet', 'comment', 'like'] 0
Index(['tweet_type', 'creator_follower_count', 'creator_following_count',
       'engager_follower_count', 'engager_following_count', 'media', 'dt_dow',
       'dt_hour', 'len_domains', 'creator_main_language',
       'engager_main_language', 'creator_and_engager_have_same_main_language',
       'is_tweet_in_creator_main_language',
       'is_tweet_in_engager_main_language',
       'engager_feature_number_of_previous_like_engagement',
       'engager_feature_number_of_previous_reply_engagement',
       'engager_feature_number_of_previous_retweet_engagement',
       'engager_feature_number_of_previous_comment_engagement',
       'number_of_engagements_positive', 'number_of_engagements_ratio_reply',
       'number_of_engagements_ratio_like',
       'number_of_engagements_ratio_retweet',
       'number_of_engagements_ratio_comment'],
      dtype='object')
Index(['tweet_type', 'creator_follower_count', 'creator_

UnboundLocalError: local variable 'test_df' referenced before assignment

## evaluate

In [9]:
path = '/hdd/twitter/raw_lzo/part-00000'
df = read_data(path)

In [10]:
df['reply_timestamp']   = df['reply_timestamp'].fillna(0)
df['retweet_timestamp'] = df['retweet_timestamp'].fillna(0)
df['comment_timestamp'] = df['comment_timestamp'].fillna(0)
df['like_timestamp']    = df['like_timestamp'].fillna(0)

df['reply_timestamp']   = df['reply_timestamp'].astype(np.uint32)
df['retweet_timestamp'] = df['retweet_timestamp'].astype(np.uint32)
df['comment_timestamp'] = df['comment_timestamp'].astype(np.uint32)
df['like_timestamp']    = df['like_timestamp'].astype(np.uint32)

df['reply'] = df['reply_timestamp'].apply(lambda x: 1 if x > 0 else 0).astype(np.int32)
df['retweet'] = df['retweet_timestamp'].apply(lambda x: 1 if x > 0 else 0).astype(np.int32)
df['comment'] = df['comment_timestamp'].apply(lambda x: 1 if x > 0 else 0).astype(np.int32)
df['like'] = df['like_timestamp'].apply(lambda x: 1 if x > 0 else 0).astype(np.int32) 

In [11]:
len(df)

3033902

In [12]:
pred = pd.read_csv('../submission/results.csv', names=['tweet_id','user_id','reply_pred','retweet_pred','comment_pred','like_pred'])

In [19]:
pred

,tweet_id,user_id,reply_pred,retweet_pred,comment_pred,like_pred
0,5DCE3020423313CB0AC2631DEF94D2F5,2AEB7AFFD36CA43C0C963F24F75BCCAC,0.007657,0.011848,0.004873,0.426737
1,2B6F5C7C4F04F34543C09DD6E58C8F66,865C52D28FA84E5E233B530C6FDAECC2,0.004492,0.005496,0.001447,0.185085
2,784D3AE5E6429392245BD34FF4C82B5C,FE83E568BD4B2ACDBAF876CFBB6C7296,0.001964,0.001596,0.002433,0.188151
3,CA49ACBB31921F2A0FAC1CCCC99541F6,DB0201BF93E4CCA1B55932AC36923E43,0.000268,0.052560,0.002369,0.082924
4,2C95B42C651B36D84584F3653D722142,04F45C4A769FE2D7C792A96C92342DC2,0.064634,0.008082,0.001255,0.604457
...,...,...,...,...,...,...
3033897,25B1D557E6B6D7298CA3888C72F9F7B5,AB51B05ACD422823A8567E51D106FBCF,0.009160,0.004641,0.000891,0.197317
3033898,3A656480D09FCCBA39B0E28FBFA64B72,2F0B0497A2C1F6DBF120782E915B1F63,0.066106,0.267093,0.001955,0.454401
3033899,E236B2359B119B2A7668FA83AC5E750B,5FFAD564CE93396A11C7951AA07D019E,0.014096,0.136655,0.001153,0.069924
3033900,B42CB5B47ABCCA8700686CD8F32696B6,1C3C0C32D7AEBCA253DC54EC85084951,0.000563,0.006479,0.001396,0.497590


In [13]:
len(pred)

3033902

In [20]:
reply_rce = compute_rce(pred['reply_pred'], df['reply'])
like_rce = compute_rce(pred['like_pred'], df['like'])
comment_rce = compute_rce(pred['comment_pred'], df['comment'])
retweet_rce = compute_rce(pred['retweet_pred'], df['retweet'])
print(reply_rce, like_rce, comment_rce, retweet_rce)

33.69630837148914 14.853137648769966 24.577383676693533 28.64781669035882


In [22]:
reply_ap = average_precision_score(df['reply'], pred['reply_pred'])
like_ap = average_precision_score(df['like'], pred['like_pred'])
comment_ap = average_precision_score(df['comment'], pred['comment_pred'])
retweet_ap = average_precision_score(df['retweet'], pred['retweet_pred'])
print(reply_ap, like_ap, comment_ap, retweet_ap)

0.33898011139657563 0.6428957690094725 0.13560077317089178 0.4173661119936234
